In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# pyUsercalc

A jupyter notebook that attempts to recreate most of the functionality of Spiegelman's [UserCalc](http://www.ldeo.columbia.edu/~mspieg/UserCalc) website for calculating Uranium Series disequilibrium calculations for the Equilbrium transport model describe in 

Spiegelman, M., 2000. UserCalc: A Web-based uranium series calculator for magma migration problems. Geochem. Geophys. Geosyst. 1, 1016. https://doi.org/10.1029/1999GC000030


## Some plotting utilities

In [ ]:
import matplotlib as mpl
mpl.rcParams['lines.linewidth']=2
mpl.rcParams['font.size']=16
## Some utility functions for plotting

def plot_inputs(df,figsize=(8,6)):
    ''' 
        pretty plots input data from pandas dataframe df
    '''
    
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True,figsize=figsize)
    ax1.plot(df['F'],df['P'])
    ax1.invert_yaxis()
    ax1.set_xlabel('F')
    ax1.set_ylabel('Pressure (kb)')
    xticks = np.linspace(0,max(df['F']),10)
    ax1.grid()
    ax1.set_xticks(xticks,minor=True)
    ax2.plot(df['Kr'],df['P'])
    ax2.set_xlabel('Kr')
    for s in ['DU','DTh','DRa','DPa']:
        ax3.semilogx(df[s],df['P'],label=s)
    ax3.set_xlabel('Ds')
    ax3.legend(loc='best',bbox_to_anchor=(1.1,1))
    
def plot_1Dcolumn(df,figsize=(8,6)):
    '''
        pretty plots output data from dataframe of output
    '''

## The main UserCalc class

This class takes in a dataframe of inputs,  initializes the appropriate splines and porosity functions
then presents a set of methods for calculating 1D columns and contour maps

In [ ]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d

class UserCalc:
    ''' A class for constructing solutions for Equilibrium Transport U-series calculations
        ala Spiegelman, 2000, G-cubed
        
        Usage: 
        us = UserCalc(df,dPdz = 0.32373, n = 2., tol=1.e-6, phi0 = 0.01) 
        
        with 
        df   :  a pandas-dataframe with columns ['P','Kr','DU','DTh','DRa','DPa']
        dPdz :  Pressure gradient to convert P to z
        n    :  permeability exponent
        tol  :  tolerance for ODE solver
        phi0 :  initial 
        
        Methods:
            
    '''
    def __init__(self, df, dPdz = 0.32373, n = 2., tol = 1.e-6, phi0 = 0.01):
        self.df = df
        self.dPdz = dPdz
        self.n = n
        self.tol = 1.e-6
        self.phi0 = phi0
        
        # set depth scale h
        self.zmin = df['P'].min()/dPdz
        self.zmax = df['P'].max()/dPdz
        self.h = self.zmax - self.zmin
        
        # lambda function to define scaled column height zprime
        self.zp = lambda P: (self.zmax - P/dPdz)/self.h
        
        # set interpolants for F and Kr and pressure
        self.F = interp1d(self.zp(df['P']),df['F'],kind='cubic')
        self.Kr = interp1d(self.zp(df['P']),df['Kr'],kind='cubic')
        self.P = interp1d(self.zp(df['P']),df['P'],kind='cubic')

        # set maximum degree of melting
        self.Fmax = self.df['F'].max()
        
        # set reference densities (assuming a mantle composition)
        self.rho_s = 3300.
        self.rho_f = 2800.
        
        
        # initialize reference permeability
        self.setAd(self.phi0,n=n)
        
        # initialize porosity function
        
    def setAd(self,phi0,n):
        ''' 
            sets the reference permeability given the maximum porosity 
        '''
        Fmax = self.Fmax
        self.Ad =  (self.rho_s/self.rho_f*Fmax - phi0*(1. - Fmax)/(1. - phi0))/(phi0**n*(1.-phi0))
        
    def phi(self,zp):
        '''
        returns porosity as function of dimensionless column height zp
        '''
        # effective permeability
        K = self.Kr(zp)*self.Ad
        
        # degree of melting
        F = self.F(zp)
        
        # density ratio
        rs_rf = self.rho_s/self.rho_f
        
        # rootfinding function to define porosity such that f(phi) = 0
        
        # check if scalar else loop
        if np.isscalar(zp):
             f = lambda phi: K*phi**self.n*(1. - phi)**2 + phi*(1. + F*(rs_rf - 1.)) - F*rs_rf
             phi = brentq(f,0.,1.)
        else: # loop over lenght of zp
            phi = np.zeros(zp.shape)
            for i,z in enumerate(zp):
                f = lambda phi: K[i]*phi**self.n*(1. - phi)**2 + phi*(1. + F[i]*(rs_rf - 1.)) - F[i]*rs_rf
                phi[i] = brentq(f,0.,1.)
        return phi
                

## Let's look at some input data

In [ ]:
df = pd.read_csv('data/sample.csv',skiprows=1,dtype=float)

In [ ]:
plot_inputs(df)

In [ ]:
# some quick tests

u = UserCalc(df)

# plot degree of melting and porosity
z = np.linspace(0,1.,100)

fig, ax1 = plt.subplots()
ax1.plot(u.phi(z),u.P(z),'r',label='$\phi$')
ax1.set_xlabel('Porosity',color='r')
ax1.set_ylabel('Pressure (kb)')
ax1.invert_yaxis()


ax2 = ax1.twiny()
ax2.plot(u.F(z),u.P(z),'b',label='$F$')
ax2.set_xlabel('Degree of melting',color='b')
plt.show()
